### <center>Hash<center>

#### <center>局部敏感hash（local sensitive hash）<center>

##### simhash
- https://blog.csdn.net/madujin/article/details/53152619
- https://blog.csdn.net/heiyeshuwu/article/details/69706414
- https://www.cnblogs.com/csj007523/p/7773027.html

In [22]:
import jieba
import jieba.analyse
import numpy as np
import jieba.posseg as pseg

In [61]:
pow(2,64)

18446744073709551616

In [62]:
def filter_nosie(line):
    filter_pool = ["。",".",";","，","|","&","%","-","=","\"","\'","“","”","(",")","（","）",":","：","\n"]
    # chinese_character = ["一", "二", "三", "四", "五", "六", "七", "八", "九", "十", "零","○"]
    # alb_number = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
    # for c in chinese_character:
    #     line = line.replace(c, "")
    # for c in alb_number:
    #     line = line.replace(c, "")
    for c in filter_pool:
        line = line.replace(c, "")
    return line.lower()

In [2]:
def string_hash(source):
    if source == "":
        return 0
    else:
        x = ord(source[0]) << 7
        m = 1000003
        mask = 2 ** 128 - 1
        for c in source:
            x = ((x * m) ^ ord(c)) & mask
        x ^= len(source)
        if x == -1:
            x = -2
        x = bin(x).replace('0b', '').zfill(64)[-64:]
        # print(source, x)

        return str(x)

In [3]:
stops_title = ["原创","转载",'音频','视频','图片']

In [14]:
def keywords2hash(keyWord):
    keyList = []
    # print(keyWord)
    for feature, weight in keyWord:
        weight = int(weight * 20)
        feature = string_hash(feature)
        temp = []
        for i in feature:
            if (i == '1'):
                temp.append(weight)
            else:
                temp.append(-weight)
        # print(temp)
        keyList.append(temp)
    list1 = np.sum(np.array(keyList), axis=0)
    if (keyList == []):  # 编码读不出来
        return '00'
    simhash = ''
    for i in list1:
        if (i > 0):
            simhash = simhash + '1'
        else:
            simhash = simhash + '0'
    return simhash,keyWord

In [59]:
def weight_distribution(l:int):
    """
    权重分配
    越靠前权重越高，总权重为1
    """
    s = [1]
    for i in range(1,l):
        c = s[i-1]
        s[i-1] = c/2
        s.append(c/2)
    return s

In [55]:
class Word(object):
    def __init__(self, token, pos, weight=0):
        self.token = token
        self.pos = pos
        self.weight = weight

filter_pos = ["c","f","uj","r","x","p"]
keep_pos = ['ns', 'n', 'nz','nrt','vn', 'v','t','tg','tg','qt','mq','m','nr','nt',"vn",'q',"eng"]
def filter_by_pos(seg_list,filter_type=0):
    if filter_type == 0:
        return [f.token for f in seg_list if f.pos not in filter_pos]
    else:
        return [f.token for f in seg_list if f.pos in keep_pos]
    
def get_simhash_shorttext(short_content):
    for s in stops_title:
        short_content = short_content.replace(s,"")
    # segs = jieba.lcut(short_content)
    words = [Word(word, tag) for word, tag in pseg.cut(short_content)]
#     print(words)
    seg_list = filter_by_pos(seg_list=words, filter_type=1)
    len_segs = len(seg_list)
    ws = weight_distribution(len_segs)
    # for word, tag in pseg.cut(short_content):
    #     print(word, tag)
    keyWord = []
    for i,seg in enumerate(seg_list):
        if len(keyWord) <=20:
            keyWord.append((seg,ws[i]))
    return keywords2hash(keyWord)
    

In [56]:
content1 = "淘宝开发者大会-20191022-003"
simhash1,keyWord1 = get_simhash_shorttext(content1)
keyWord1

[('淘宝', 0.5),
 ('开发者', 0.25),
 ('大会', 0.125),
 ('20191022', 0.0625),
 ('003', 0.0625)]

In [16]:
def get_simhash(content):
    for s in stops_title:
        content = content.replace(s,"")
    keyWord = jieba.analyse.textrank(sentence=content,
                                     topK=20,
                                     withWeight=True,
                                     allowPOS=('ns', 'n', 'nz','nrt','vn', 'v','t','tg','tg','qt','mq','m','nr','nt',"vn",'q'))

    if len(keyWord) == 0:
        segs = jieba.lcut(content)
        for seg in segs:
            if len(keyWord) <=20:keyWord.append((seg,0.5))
    return keywords2hash(keyWord)

In [17]:
# long text
content = "蒲公英-采用全新智能组网技术的路由器-20191022"
simhash,keyWord = get_simhash(content)
simhash

# '0010101010001010111011001100101110001001011101110001010000111101'
# '0010101010001000111010001101110010001101001001100000010010011111'

'0010101010001000111010001101110010001101001001100000010010011111'

In [25]:
# short text
content = "蒲公英-采用全新智能组网技术的路由器-20191022"
simhash,keyWord = get_simhash_shorttext(content)
simhash

'0110101011001000111010001101110010001101001001100011010010011111'

In [32]:
keyWord

[('蒲公英', 0.5),
 ('采用', 0.5),
 ('智能', 0.5),
 ('组网', 0.5),
 ('技术', 0.5),
 ('路由器', 0.5),
 ('20191022', 0.5)]

In [29]:
def hammingDis(com,simhash):
    """
    汉明距离
    """
    t1 = '0b' + simhash
    if isinstance(com, str):
    # if type(com) == str:
        t2 = '0b' + com
    else:
        t2 = '0b' + com.simhash
    n = int(t1, 2) ^ int(t2, 2)
    i = 0
    while n:
        n &= (n - 1)
        i += 1
    return i

In [60]:
content1 = "淘宝开发者大会-20191022-001"
content2 = "淘宝开发者大会-20191022-002"
simhash1,keyWord1 = get_simhash(content1)
simhash2,keyWord2 = get_simhash_shorttext(content2)
hammingDis(simhash1,simhash2)

14

In [43]:
keyWord1

[('淘宝', 0.5), ('开发者', 0.5), ('大会', 0.5), ('20191022', 0.5), ('001', 0.5)]

In [41]:
def find_dup(coms,simhash,distance):
    """
    dup 重复True
    """
    if simhash in coms:
        return True
    
    if distance != 0:
        for com in coms:
            if hammingDis(com,simhash) <= distance:
                """存在重复文件"""
                return True
    # coms.append(self.simhash)
    return False

In [48]:
coms = []
coms.append(simhash)
content = "蒲公英-采用全新智能组网技术的路由器-2019-10-22"
simhash1,keyWord = get_simhash(content)
# simhash1 = '0010101010001000111010001101110010001101001001100000010010011110'
find_dup(coms, simhash1, 4)

False

In [16]:
content1 = "通信行业：地方政府助推5G建设，国务院大力发展智慧交通，持续看好5G板块－跟踪周报20190921" + "投资要点 \n     策略观点:1、各省市陆续发布5G发展计划,深圳表示2020年8月底将累计建成5G基站4.5万个,5G基站建设密度全国领先。基于5G通讯技术推动商用产业发展,是目前北京、上海、广州、深圳一线城市及全国多地出台发展规划中的重中之重。我国网络建设及终端先于应用板块崛起,同时随着5G终端产业加速,5G终端板块投资机会逐步显现。2、国务院印发《交通强国建设纲要》,大力发展智慧交通,5G作为端-管-云之间的衔接桥梁,助力构建车路云协同的新型交通体系。3、个股方面建议关注两条主线:5G网络建设及5G应用及流量,从投资节奏来看,5G网络建设先行,流量及应用在网络建设之后,弹性更大。 \n     行业动态及点评:各地陆续推出5G政策,统筹加快5G建设,提升5G产业链协同创新与集聚能力,助力经济高质量发展。国务院印发《交通强国建设纲要》,大力发展智慧交通,上下游企业资本信心更足,投入更加坚定。美的、中国电信、华为三方联合发布《5G+智能工厂网络及应用白皮书》,加快5G+工业互联网领域研究探索及项目落地。\n      行业前瞻:2019亚太内容分发大会(2019年9月24日-25日)近期推荐组合:1、5G网络建设:中兴通讯(000063)、中国联通(600050)、世嘉科技(002796)、烽火通信(600498)、中际旭创(300308)。2、5G应用和流量:淳中科技(603516)、中科创达(300496)、二六三(002467)、中新赛克(002912)。中国铁塔(港股0788),网络规划设计板块相关个股也值得关注。\n      建议关注组合:1、5G网络建设:中石科技(300684)、沪电股份(002463)、深南电路(002916)、新易盛(300502)、光迅科技(002281)、俊知集团(港股1300)。2、5G应用和流量:网宿科技(300017)、紫光股份(000938)、移为通信(300590)、会畅通讯(300578)、高新兴(300098)。3、云计算基础设施:光环新网(300383)、宝信软件(600845)、数据港(603881)。4、智慧交通:佳都科技(600728)、中科创达(300496)、四维图新(002405)、高新兴(300098)、宜通世纪(300310)。 \n     市场回顾:近一周通信(申万)指数下跌1.72％；沪深300指数下跌0.92％；行业落后大盘0.80％。 \n     风险提示:中美贸易摩擦缓和低于预期风险,行业增速放缓风险\n \n"

In [26]:
simhash1,keyWord = get_simhash(content1)
simhash1

'0010101110100001111010111100101011000010001010100011101111110100'

In [18]:
content2 = "通信行业：地方政府助推5G建设，国务院大力发展智慧交通，持续看好5G...20190921" + "投资要点\n     策略观点:1、各省市陆续发布5G发展计划,深圳表示2020年8月底将累计建成5G基站4.5万个,5G基站建设密度全国领先。基于5G通讯技术推动商用产业发展,是目前北京、上海、广州、深圳一线城市及全国多地出台发展规划中的重中之重。我国网络建设及终端先于应用板块崛起,同时随着5G终端产业加速,5G终端板块投资机会逐步显现。2、国务院印发《交通强国建设纲要》,大力发展智慧交通,5G作为端-管-云之间的衔接桥梁,助力构建车路云协同的新型交通体系。3、个股方面建议关注两条主线:5G网络建设及5G应用及流量,从投资节奏来看,5G网络建设先行,流量及应用在网络建设之后,弹性更大。\n     行业动态及点评:各地陆续推出5G政策,统筹加快5G建设,提升5G产业链协同创新与集聚能力,助力经济高质量发展。国务院印发《交通强国建设纲要》,大力发展智慧交通,上下游企业资本信心更足,投入更加坚定。美的、中国电信、华为三方联合发布《5G+智能工厂网络及应用白皮书》,加快5G+工业互联网领域研究探索及项目落地。\n     行业前瞻:2019亚太内容分发大会(2019年9月24日-25日)近期推荐组合:1、5G网络建设:中兴通讯(000063)、中国联通(600050)、世嘉科技(002796)、烽火通信(600498)、中际旭创(300308)。2、5G应用和流量:淳中科技(603516)、中科创达(300496)、二六三(002467)、中新赛克(002912)。中国铁塔(港股0788),网络规划设计板块相关个股也值得关注。\n     建议关注组合:1、5G网络建设:中石科技(300684)、沪电股份(002463)、深南电路(002916)、新易盛(300502)、光迅科技(002281)、俊知集团(港股1300)。2、5G应用和流量:网宿科技(300017)、紫光股份(000938)、移为通信(300590)、会畅通讯(300578)、高新兴(300098)。3、云计算基础设施:光环新网(300383)、宝信软件(600845)、数据港(603881)。4、智慧交通:佳都科技(600728)、中科创达(300496)、四维图新(002405)、高新兴(300098)、宜通世纪(300310)。\n     市场回顾:近一周通信(申万)指数下跌1.72％；沪深300指数下跌0.92％；行业落后大盘0.80％。\n     风险提示:中美贸易摩擦缓和低于预期风险,行业增速放缓风险。\n \n"

In [27]:
simhash2,keyWord = get_simhash(content2)
simhash2

'0010001110100001111010111100101011000010001010100011101111110100'

In [31]:
s1 = "0010001100000101111010111001111011010010001010100011110111100101"
s2 = "0010001100000101111010110001111011010010001000101011100111100001"
hammingDis(s1,s2)

5

In [46]:
keyWord

[('路由器', 1.0),
 ('智能', 0.9904208972881985),
 ('技术', 0.9759882078368103),
 ('2019', 0.8464088764053259),
 ('采用', 0.8154838850302057),
 ('组网', 0.7878571377272482),
 ('10', 0.6611890618238442),
 ('22', 0.47140543597476264),
 ('蒲公英', 0.43795179996834765)]

### cpython-simhash

In [3]:
import re
from simhash import Simhash, SimhashIndex

In [2]:
def get_features(s):
    width = 3
    s = s.lower()
    s = re.sub(r'[^\w]+', '', s)
    return [s[i:i + width] for i in range(max(len(s) - width + 1, 1))]
get_features('How are you?')

['how', 'owa', 'war', 'are', 'rey', 'eyo', 'you']

In [5]:
data = {
    1: u'How are you? I Am fine. blar blar blar blar blar Thanks.',
    2: u'How are you i am fine. blar blar blar blar blar than',
    3: u'This is simhash test.',
}
objs = [(str(k), Simhash(get_features(v))) for k, v in data.items()]
index = SimhashIndex(objs, k=3)  # `k` is the tolerance
index.bucket_size

<bound method SimhashIndex.bucket_size of <simhash.SimhashIndex object at 0x7f4f39e79550>>

In [6]:
s1 = Simhash(get_features(u'How are you i am fine. blar blar blar blar blar thank'))
print (index.get_near_dups(s1))

['1']


In [8]:
s1.value

7604580641891645972

In [9]:
index.add('4', s1)
print (index.get_near_dups(s1))

['1', '4']


#### md5
MD5即Message-Digest Algorithm 5（信息-摘要算法5）

In [49]:
import hashlib

def get_md5(text):
    m = hashlib.md5()
    m.update(text.encode(encoding='UTF-8'))
    return m.hexdigest()
get_md5("我是一个词")

'6a360e983a12c112479ff97581f5723c'

### perceptual hash
https://github.com/knjcode/imgdupes
Perceptual hashes are a different concept compared to cryptographic hash functions like MD5 and SHA1.
均值hash
pHash
https://www.cnblogs.com/faith0217/articles/4088386.html